In [ ]:
from component.GeracaoMalhaHexagonalComponent import GeracaoMalhaHexagonalComponent
from model.constants.ParametrosConstantes import ParametrosConstantes
from service.MunicipioService import MunicipioService
from util.BancoDadosUtil import BancoDadosUtil
from util.DataFrameUtil import DataFrameUtil
from util.LoggerUtil import LoggerUtil
from util.WarningUtil import WarningUtil

from pandas import DataFrame
from time import sleep

In [ ]:
# Configurações iniciais

WarningUtil.ignorar_warning(categoria=DeprecationWarning)
WarningUtil.ignorar_warning(categoria=UserWarning)

log = LoggerUtil.configurar_logger(arquivo_log=ParametrosConstantes.CAMINHO_LOG_GERACAO_MALHA_HEXAGONAL)

In [ ]:
log.info(msg="Iniciando o processo de geração da malha hexagonal dos municípios.")

conexao_bd = BancoDadosUtil.estabelecer_conexao_banco()

geracao_malha_hexagonal_component = GeracaoMalhaHexagonalComponent()
municipio_service = MunicipioService()

In [ ]:
while True:
    if geracao_malha_hexagonal_component.verificar_nao_existencia_registros_pendentes(conexao_bd):
        break

    log.info(msg=f"Buscando os próximos {ParametrosConstantes.BATCH_QTDE_REGISTROS_ETAPA_GERACAO_MALHA} municípios a serem processados.")

    gdf_municipio = municipio_service.buscar_registros_pendentes_geracao_malha_hexagonal(conexao_bd)

    log.info(msg="Gerando a malha hexagonal dos municípios.")

    df_resultado = DataFrameUtil.processar_dataframe_dask(
        df=gdf_municipio, 
        funcao=geracao_malha_hexagonal_component.processar_particao_dask, 
        meta=DataFrame(columns=["codigo_municipio", "lista_dict_malha_hexagonal", "lista_dict_historico_erro", "status"]),
        qtde_particoes=ParametrosConstantes.BATCH_QTDE_PARTICOES_DASK_ETAPA_GERACAO_MALHA
    )

    geracao_malha_hexagonal_component.persistir_resultado(df_resultado, conexao_bd)
    sleep(10)

In [ ]:
BancoDadosUtil.encerrar_conexao_banco(conexao_bd)